# Accessing PubChem through PUG-View

```{dropdown} About this interactive ![icons](../static/img/rocket.png) recipe
- Author(s): [Sunghwan Kim](https://orcid.org/0000-0001-9828-2074)
- Reviewer: [Samuel Munday](https://orcid.org/0000-0001-5404-6934)
- Topic(s): How to retrieve chemical data using the PubChem View API.
- Format: Interactive Jupyter Notebook (Python)
- Scenario: You need to access and chemical data programatically based from PubChem annotations.
- Skills: You should be familar with:
    - [Application Programming Interfaces (APIs)](https://www.ibm.com/topics/api)
    - [Representational State Transfer API](https://restfulapi.net/)
    - [Introductory Python](https://www.youtube.com/watch?v=kqtD5dpn9C8)
- Learning outcomes:
    - How to get PubChem annotations for a compound
    - How to get PubChem annotations of multi-word sections and chemical properties for a compound
    - How to get PubChem annotations for data other types of records
- Citation: 'Accessing PubChem through PUG-View', Sunghwan Kim, The IUPAC FAIR Chemistry Cookbook, Contributed: 2024-02-14 [https://w3id.org/ifcc/IFCC009](https://w3id.org/ifcc/IFCC009).
- Reuse: This notebook is made available under a [CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/) license.
```

PubChem provides multiple programmatic access routes.  One of them is PUG-View, which is a Representational State Transfer (REST)-style web service interface specialized for accessing annotation data contained in PubChem.  More detailed information on PUG-View can be found in the following paper:

>**PUG-View: programmatic access to chemical annotations integrated in PubChem**  
>Kim et al., Journal of Cheminformatics, 2019, 11:56.  
>DOI: [10.1186/s13321-019-0375-2](https://doi.org/10.1186/s13321-019-0375-2)

Additional information can also be found at the PUG-View Help page ([https://pubchem.ncbi.nlm.nih.gov/docs/pug-view](https://pubchem.ncbi.nlm.nih.gov/docs/pug-view).

This Jupyter notebook demonstrates how to programmatically access annotation data for PubChem records.

In [ ]:
import requests
import time
import json

## 1. Annotations for a PubChem compound record.

PubChem collects a wide range of information on chemicals and presents it on Compound Summary pages.

Let's suppose that we want to find the viscosity of dimethyl sulfoxide (DMSO; CID 679).  With a web browser like Google Chrome or MS Edge, you can find this information on the following web page:

[https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Viscosity](https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Viscosity)

![Viscosity of CID 679 (DMSO)](../images/pubchem_pugview_fig1.png)

The viscosity of DMSO is 2.47 cP at 20 &deg;C.  It is expressed in cP (centipoise). 1 cP is equivalent to 0.01 P (Poise).

Now we want to get this viscosity information programmatically, using PUG-View.

In [ ]:
# Formulate a PUG-View request URL.
base = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view"
cid = "679"
output = "JSON"
heading = "Viscosity"

url = base + "/data/compound/" + cid + "/" + output + "?heading=" + heading
print(url)   # Check the request URL

Click the above PUG-View request URL to view the requested data in a web browser.

The request URL can also be used in a python script.  The following cell sends a PUG-View request to the PubChem server and prints the returned data in a json format.

In [ ]:
# Get the viscosity data and store it as "mydata"
mydata = requests.get(url).json()
print(json.dumps(mydata, indent=4))

JSON-formatted data downloaded through PUG-View is often very large and not very easy to read.  Therefore, it is necessary to **parse** the JSON-formatted data to extract necessary pieces of information, as shown in the next few cells.

In [ ]:
node = mydata['Record']['Section'][0]['Section'][0]['Section'][0]['Information'][0]
node

In [ ]:
value = node['Value']['StringWithMarkup'][0]['String']
reference = node['Reference'][0]

In [ ]:
print(value)

In [ ]:
print(reference)

There are many other annotation data available for this compound (DMSO). Here are some examples of them.

In [ ]:
base = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view"
cid = "679"
output = "JSON"
headings = ["Density", "Decomposition", "Solubility", "Odor"]

for heading in headings:
    url = base + "/data/compound/" + cid + "/" + output + "?heading=" + heading
    print(url)   # Check the request URL

The above PUG-View requests return the data presented on the corresponding sections of the compound summary page of DMSO (CID 679)
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Density  
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Decomposition  
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Solubility  
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Odor  


It is noteworthy that the ```heading``` parameter (after the "?" character) in a PUG-View request URL is optional.  When the ```heading``` parameter is not given, all annotations available for the compound will be returned.  For example, the following request URL returns all annotation data for DMSO (CID 679).

In [ ]:
url = base + "/data/compound/" + cid + "/" + output  # No heading parameter is set.  All annotations for the CID will be returned.
print(url)   # Check the request URL

## 2. Data under a  multi-word section heading

Some section headings consist of multiple words (e.g., "Vapor Pressure"). For example, the following screenshot shows the vapor pressure of CID 679 (DMSO), which can be accessed at:  
https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Vapor-Pressure


![Vapor pressure of CID 679 (DMSO)](../images/pubchem_pugview_fig2.png)

To download the data presented under a multi-word section heading, the space between the words in the heading should be replaced with the "+" character, as shown in the following example:

In [ ]:
# Formulate a PUG-View request URL.
base = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view"
cid = "679"
output = "JSON"
heading = "Vapor+Pressure"

url = base + "/data/compound/" + cid + "/" + output + "?heading=" + heading
print(url)   # Check the request URL

In [ ]:
# Get the viscosity data and store it as "mydata"
mydata = requests.get(url).json()
#print(json.dumps(mydata, indent=4))  # uncomment this line to print the data.

Data downloaded through PUG-View can be saved.

In [ ]:
with open("../files/cid679-vapor-pressure.json", "w") as outfile:
    json.dump(mydata, outfile, indent=4)

Many heading names consist of multiple words.  Here are some examples \[for DMSO (CID 679)\].

In [ ]:
base = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view"
cid = "679"
output = "JSON"
headings = ["Autoignition+Temperature", 
            "Refractive+Index",
            "Exposure+Routes",
            "Non-Human+Toxicity+Values",
            "Ecotoxicity+Excerpts"]

for heading in headings:
    url = base + "/data/compound/" + cid + "/" + output + "?heading=" + heading
    print(url)   # Check the request URL

The returned annotations are also presented in the compound summary page of DMSO (CID 679).
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Autoignition-Temperature  
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Refractive-Index  
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Exposure-Routes
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Non-Human-Toxicity-Values
- https://pubchem.ncbi.nlm.nih.gov/compound/679#section=Ecotoxicity-Excerpts


## 3. Getting All Annotation Data for a Given Property

In some cases, you may want to download all annotation data for a given property (for example, all human-toxicity values or all boiling-point values available in PubChem).  This can be readily done through PUG-View, using the following request URLs.

### 3.1 Annotations for single-word headings

In [ ]:
heading = "Viscosity"

In [ ]:
# The heading is included in the URL query string.
url = base + "/annotations/heading/JSON?heading=" + heading
print(url)

In [ ]:
# The heading is included in the URL path.
url = base + "/annotations/heading/" + heading + "/JSON"
print(url)

The above two request URLs differ in where the heading name appears.  That is, the heading name can be encoded in:
- the URL query string (after the "?" character), as in the first URL.
- the URL path, as in the second URL.

The two URLs do exactly the same thing when the heading name consists of a single word.

### 3.2 Annotations for multiple-word headings

When the heading name consists of multiple words, it is highly recommended to provide the heading name in the query string, although it is possible to provide it in the URL path. Let's consider the task of downloading all Kovats retention index values.

In [ ]:
heading1 = "Kovats%20Retention%20Index"  # spaces are replaced with "%20"
heading2 = "Kovats+Retention+Index"      # spaces are replaced with "+"

In [ ]:
url1 = base + "/annotations/heading/JSON?heading=" + heading1
url2 = base + "/annotations/heading/JSON?heading=" + heading2
print(url1)
print(url2)

In [ ]:
# The heading is included in the URL path.
url1 = base + "/annotations/heading/" + heading1 + "/JSON"
url2 = base + "/annotations/heading/" + heading2 + "/JSON"
print(url1)
print(url2)    #-- This does not work!

When a multi-word heading name is provided in the URL query string (after the "&" character), the words can be separated by "%20" or "+" characters.  However, only "%20" (not "+") can be used when the heading name is provided in the URL path.  Because this subtle difference is somewhat confusing, it is highly recommended that the heading name be always provided in the query string, which makes the request URL easier to read.

### 3.3 Multi-page annotations

A PUG-View request has a maximum time limit of 30 seconds. If a PUG-View request exceeds this limit, a time-out error is returned. However, some headings have a large number of annotation data, making it impossible to get them within 30 seconds through a single PUG-REST request.  To circumvent this issue, the annotation data are **paginated**.

The data returned from a PUG-View request contains the pagination information, which can be found at the end of the returned json data. It looks similar to this:

![PUG-View-Pagination](../images/pubchem_pugview_fig3.png)

The pagination information includes the total number of pages for the requested data and the page number returned from the current request.  Therefore, the above image indicates that the requested annotation data are 80-pages-long, and only the first page has been returned from the current request.  To get all the data, you should loop through all pages, as shown in the following example.

In [ ]:
heading = "Kovats+Retention+Index"      # spaces are replaced with "+"

num_page_total = 80
data = [None] * num_page_total

for i in range(num_page_total):
    
    page_curr = i + 1
    url = base + "/annotations/heading/JSON?heading=" + heading + "&page=" + str(page_curr)
    data[i] = requests.get(url).json()
    
    if page_curr % 5 == 0 :
        print("Getting data for page", page_curr)
    
    time.sleep(0.2)

In this example, the downloaded data are stored as a list of json objects, while the data for each page can be stored in a separate variable or file.  Also note that **time.sleep(0.2)** is added to avoid overloading PubChem servers.  According to [PubChem's usage policy](https://pubchem.ncbi.nlm.nih.gov/docs/programmatic-access), the user should not submit more than 5 requests per second and violators will be blocked for a certain period of time (typically less than 24 hours).

### 3.4 Annotations from a specific data collection

PubChem has multiple data collections, including Substance, Compound, BioAssay, Gene, Protein, Pathway, Cell Line, Taxonomy, Element, and Patent.  Some annotation headings appear in more than one data collection.  For example, both a compound and an element can have the "boiling point" information, as shown in these examples:

- Compound (benzene as an example)  
https://pubchem.ncbi.nlm.nih.gov/compound/241#section=Boiling-Point

- Element (argon as an example)  
https://pubchem.ncbi.nlm.nih.gov/element/18#section=Boiling-Point

Therefore, when downloading annotations for a heading used in multiple data collections, it is necessary to specify what data collections you want to download data from.  This can be done using the "heading_type" parameter.

In [ ]:
# Getting annotations for compounds
heading = "Boiling+Point"      # spaces are replaced with "+"
url = base + "/annotations/heading/JSON?heading=" + heading + "&heading_type=compound"
print(url)

In [ ]:
# Getting annotations for elements
heading = "Boiling+Point"      # spaces are replaced with "+"
url = base + "/annotations/heading/JSON?heading=" + heading + "&heading_type=element"
print(url)

Note that, in the above two URL examples, the ```heading_type``` parameter is set to "compound" and "element" to download boiling points for compounds and elements, respectively.

## 4. Annotations for PubChem records other than compounds.

With PUG-View, you can programmatically access annotations for compounds as well as other PubChem records, such as genes, proteins, pathways, cell lines, taxons, and patents.  
The examples below demonstrate how to access the data presented on the following summary pages:
- https://pubchem.ncbi.nlm.nih.gov/gene/1956#section=Protein-Isoforms
- https://pubchem.ncbi.nlm.nih.gov/protein/P10828#section=Related-Proteins
- https://pubchem.ncbi.nlm.nih.gov/cell/30#section=Diseases
- https://pubchem.ncbi.nlm.nih.gov/taxonomy/9986#section=Names-and-Identifiers
- https://pubchem.ncbi.nlm.nih.gov/patent/US-8148324-B2#section=Important-Dates

In [ ]:
# Formulate a PUG-View request URL.
base = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view"

In [ ]:
# Annotations for a gene record
record_type = "gene"
record_id = "1956"              #-- NCBI Gene ID for human EGFR
heading = "Protein+Isoforms"

url = base + "/data/" + record_type + "/" + record_id + "/JSON?heading=" + heading
print(url)

In [ ]:
# Annotations for a protein record
record_type = "protein"
record_id = "P10828"            #-- NCBI Protein Accession for human thyroid hormone receptor beta
heading = "Related+Proteins"

url = base + "/data/" + record_type + "/" + record_id + "/JSON?heading=" + heading
print(url)

In [ ]:
# Annotations for a cell line record
record_type = "cell"
record_id = "hela"              #-- Cell Line Abbreviation
heading = "Diseases"

url = base + "/data/" + record_type + "/" + record_id + "/JSON?heading=" + heading
print(url)

In [ ]:
# Annotations for a taxonomy (organism) record
record_type = "taxonomy"
record_id = "9986"              #-- NCBI Taxonomy ID for Oryctolagus cuniculus (rabbit)
heading = "Names+and+Identifiers"

url = base + "/data/" + record_type + "/" + record_id + "/JSON?heading=" + heading
print(url)

In [ ]:
# Annotations for a taxonomy (organism) record
record_type = "patent"
record_id = "US-8148324-B2"              #-- Patent ID
heading = "Important+Dates"

url = base + "/data/" + record_type + "/" + record_id + "/JSON?heading=" + heading
print(url)

As mentioned previously, the ```heading``` parameter is optional.  If it is not provided in the above request URLs, all annotation data for the records will be returned.

In addition, PUG-View can also be used to retrieve data for PubChem substance and assay records.  

In [ ]:
# Annotations for a PubChem substance record
record_type = "substance"
record_id = "319892613"              #-- PubChem Substance ID (SID)

url = base + "/data/" + record_type + "/" + record_id + "/JSON"
print(url)

In [ ]:
# Annotations for a PubChem bioassay record
record_type = "bioassay"
record_id = "720659"              #-- PubChem BioAssay ID (AID)

url = base + "/data/" + record_type + "/" + record_id + "/JSON"
print(url)

Note that PubChem Substance and BioAssay collections are archival in nature.  It means that the data for the records in these two collections are submitted by data depositors (not collected by PubChem).